In [61]:
import pandas as pd
from sklearn import metrics
import numpy as np


trainBrut = pd.read_csv("train.csv")

train = trainBrut.drop(columns=['Name', 'Cabin', 'Ticket'])

features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
featuresToDis = ['Age', 'Fare']

train['hyp'] = 0

testBrut = pd.read_csv("test.csv")
test = testBrut.drop(columns=['Name', 'Cabin', 'Ticket'])
test['Survived'] = 0

In [68]:
def stats2(test):

    print(metrics.classification_report(test.Survived, test.hyp, target_names = ['Dead', 'Alive'], zero_division=1))


In [63]:
def discretize(data, means, featuresToDiscretize):
    
    dataDiscret = data.copy()
    
    for col in featuresToDiscretize:
        for i in data.index:
            
            if (data[col][i] >= means[col]):
                dataDiscret[col][i] = "-"
            else:
                dataDiscret[col][i] = "+"
                
    return dataDiscret


trainDis = discretize(train, train.mean(), featuresToDis)

testDis = discretize(test, test.mean(), featuresToDis)

C:\Users\romai\AppData\Local\Temp\ipykernel_9492\2053728321.py:16: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  trainDis = discretize(train, train.mean(), featuresToDis)
C:\Users\romai\AppData\Local\Temp\ipykernel_9492\2053728321.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataDiscret[col][i] = "+"
C:\Users\romai\AppData\Local\Temp\ipykernel_9492\2053728321.py:18: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only vali

In [64]:
def trainOneR(train, featuresColumns):
    
    bestPrecision = -1

    for col in featuresColumns:
        res = pd.DataFrame(data = np.zeros((len(train.Survived.unique()), len(train[col].unique()))), columns = train[col].unique())

        for i in train.index:
            res[train[col][i]][int(train.Survived[i])] += 1
        
        precision = res.max().sum() / res.sum().sum()

        #Update du model avec nouvelle colone
        if (precision > bestPrecision):
            bestPrecision = precision
            model = {'feature_name': col}
            
            for value in res:
                print()
                model[value] = res.idxmax()[value]

    return model

In [65]:
def testOneR(test, model):
    for i in test.index:
        test.hyp[i] = int(model[ test[ model['feature_name'] ][i] ])

In [137]:
model = trainOneR(trainDis, featuresToDis)

print(model)

testOneR(trainDis, model)

stats2(trainDis)





{'feature_name': 'Fare', '+': 0, '-': 1}
              precision    recall  f1-score   support

        Dead       0.68      0.85      0.76       549
       Alive       0.60      0.37      0.46       342

    accuracy                           0.66       891
   macro avg       0.64      0.61      0.61       891
weighted avg       0.65      0.66      0.64       891



C:\Users\romai\AppData\Local\Temp\ipykernel_9492\143965923.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.hyp[i] = int(model[ test[ model['feature_name'] ][i] ])


In [138]:
def trainNaiveBayes(train, featuresColumns):
    
    featuresRes = {}
    bayesProbs = {}
    classesProbs = {}

    for c in train.Survived.unique():

        classesProbs[c] = pd.value_counts(train.Survived)[c] / train.Survived.count()

    for col in featuresColumns:
        res = pd.DataFrame(data = np.ones((len(train.Survived.unique()), len(train[col].unique()))), columns = train[col].unique())
        
        for i in train.index:
            res[train[col][i]][int(train.Survived[i])] += 1
        
        featuresRes[col] = res

    for feature in featuresRes.keys():
        resProbs = featuresRes[feature].copy()

        for ind in featuresRes[feature].index:
            for fValueName in train[feature].unique():
                resProbs[fValueName][ind] = featuresRes[feature][fValueName][ind] / featuresRes[feature].sum(1)[ind]
        bayesProbs[feature] = resProbs


    model = {'featuresProbs': bayesProbs, 'classesProbs': classesProbs}

    return model

In [150]:
def testNaiveBayes(test, model):

    for i in test.index:

        probs = {}
        for c in model['classesProbs'].keys():

            prob=1
            for f in model['featuresProbs'].keys():
                if test[f][i] in model['featuresProbs'][f]:
                    prob *=   model['featuresProbs'][f][test[f][i]][c]

            prob *= model['classesProbs'][c]
            probs[c] = prob



        test.hyp[i] = max(probs, key=probs.get)

    print('ok')
    


In [151]:
print(trainDis)

model = trainNaiveBayes(trainDis, features)

testNaiveBayes(trainDis, model)

trainDis

     PassengerId  Survived  Pclass     Sex Age  SibSp  Parch Fare Embarked  \
0              1         0       3    male   +      1      0    +        S   
1              2         1       1  female   -      1      0    -        C   
2              3         1       3  female   +      0      0    +        S   
3              4         1       1  female   -      1      0    -        S   
4              5         0       3    male   -      0      0    +        S   
..           ...       ...     ...     ...  ..    ...    ...  ...      ...   
886          887         0       2    male   +      0      0    +        S   
887          888         1       1  female   +      0      0    +        S   
888          889         0       3  female   +      1      2    +        S   
889          890         1       1    male   +      0      0    +        C   
890          891         0       3    male   -      0      0    +        Q   

     hyp  
0      0  
1      1  
2      0  
3      1  
4      0

C:\Users\romai\AppData\Local\Temp\ipykernel_9492\1556195326.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.hyp[i] = max(probs, key=probs.get)


ok


,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,hyp
0,1,0,3,male,+,1,0,+,S,0
1,2,1,1,female,-,1,0,-,C,1
2,3,1,3,female,+,0,0,+,S,0
3,4,1,1,female,-,1,0,-,S,1
4,5,0,3,male,-,0,0,+,S,0
...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,male,+,0,0,+,S,0
887,888,1,1,female,+,0,0,+,S,1
888,889,0,3,female,+,1,2,+,S,1
889,890,1,1,male,+,0,0,+,C,0


In [152]:
stats2(trainDis)

              precision    recall  f1-score   support

        Dead       0.80      0.85      0.82       549
       Alive       0.72      0.65      0.69       342

    accuracy                           0.77       891
   macro avg       0.76      0.75      0.75       891
weighted avg       0.77      0.77      0.77       891



In [153]:
def distMan(dataX, dataY, featuresNames):
    dist = 0
    for f in featuresNames:
        dist += np.abs(dataY[featuresNames][0] - dataX[featuresNames][0])

    return dist

In [154]:
from heapq import nsmallest

def KPPV(test, train, featuresNames, k):

    for testInd in test.index:
        distances = {}
        for trainInd in train.index:
            distances[trainInd] = distMan(test.iloc[testInd], train.iloc[trainInd], featuresNames)
        
        closestIndex = nsmallest(k, distances, key = distances.get)
        
        closestClasses = []

        for trainInd in closestIndex:
            closestClasses.append(train.iloc[trainInd].target)
        test.hyp[testInd] = max(closestClasses,key=closestClasses.count)

In [ ]:
KPPV(testIris, trainIris, iris['feature_names'], 8)

test

In [149]:
testDis.to_csv('res.csv', columns=['PassengerId', 'Survived'])